In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#preprocessing the images
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])  #mean and std

In [3]:
training_data=torchvision.datasets.CIFAR100(root = './data',
                                            train=True,
                                            download=True,
                                            transform=transform
)
testing_data=torchvision.datasets.CIFAR100(root = './data',
                                            train=False,
                                            download=True,
                                            transform=transform
)

100%|██████████| 169001437/169001437 [00:05<00:00, 29174533.62it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
batch_size=64

train_dataloader=torch.utils.data.DataLoader(training_data,batch_size=batch_size)
test_dataloader=torch.utils.data.DataLoader(testing_data,batch_size=batch_size)


In [5]:
#Architecture

class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()

    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.flatten = nn.Flatten(start_dim=1)
    self.fc1 = nn.Linear(1600,200)
    self.fc2 = nn.Linear(200,160)
    self.fc3 = nn.Linear(160,120)
    self.fc4 = nn.Linear(120,100)

  def forward(self,x):
    x = self.conv1(x) #i/p:3,32*32     o/p:6,28*28
    #x = F.relu(x)
    x = self.pool(x) # coverting 6,14*14
    #print(x.shape)
    x = self.conv2(x) #16*10*10
    #print(x.shape)
    x = self.flatten(x)
    #print(x.shape)
    x = self.fc1(x)
    #print(x.shape)
    x = F.relu(x)
    #print(x.shape)
    x = self.fc2(x)
    #print(x.shape)
    x = F.sigmoid(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    #print(x.shape)
    return x


In [6]:
#model=CNN()

In [7]:
'''
for X,y in train_dataloader:
  print(X.shape)
  model(X)
  break
'''

'\nfor X,y in train_dataloader:\n  print(X.shape)\n  model(X)\n  break\n'

In [8]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')

In [9]:
model=CNN()
model.to(device)
model


CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1600, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=160, bias=True)
  (fc3): Linear(in_features=160, out_features=120, bias=True)
  (fc4): Linear(in_features=120, out_features=100, bias=True)
)

In [10]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-2)

In [11]:
def training(train_dataloader,model,loss_,optimizer):
  model.train()
#train_loss = 0.0
#pass every batch of data to the model
  for batch, (X,y) in enumerate(train_dataloader):
    X = X.to(device)
    y = y.to(device)

    pred = model(X)
    #print(pred)
    #print(y)
    loss = loss_fn(pred,y)
    #print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 ==0:
      print(f'loss:{loss.item()}')




In [12]:
def testing(test_dataloader,model,loss_):
  model.eval()
  test_loss = 0.0
  correct = 0
  for X,y in test_dataloader:
    X = X.to(device)
    y = y.to(device)

    pred = model(X)
    #print(pred)
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss += loss_fn(pred,y).item()
  test_loss = test_loss/len(test_dataloader)
  print (len(test_dataloader.dataset))
  accuracy = (correct/len(test_dataloader.dataset))*100
  print (f'Accuracy : {accuracy} with test error {test_loss}')
    #print(correct)

In [13]:
epochs = 9 #how many times you have to send data to the model

for t in range(epochs):
  training(train_dataloader,model,loss_fn,optimizer)
  testing(test_dataloader,model,loss_fn)
  print(f' ==================== EPOCH : {t} ============================')

loss:4.606820583343506
loss:4.5964508056640625
loss:4.619381427764893
loss:4.614006042480469
loss:4.613214492797852
loss:4.600072860717773
loss:4.6163225173950195
loss:4.614313125610352
10000
Accuracy : 1.0 with test error 4.606187055065374
 ==================== EPOCH : 0 ============================
loss:4.602466583251953
loss:4.6002092361450195
loss:4.6087470054626465
loss:4.610718727111816
loss:4.606064319610596
loss:4.603142738342285
loss:4.614599704742432
loss:4.6095147132873535
10000
Accuracy : 1.0 with test error 4.605169864217187
 ==================== EPOCH : 1 ============================
loss:4.602540016174316
loss:4.600725173950195
loss:4.606535911560059
loss:4.608785629272461
loss:4.604031562805176
loss:4.604186058044434
loss:4.6130218505859375
loss:4.606669902801514
10000
Accuracy : 1.03 with test error 4.604340993674698
 ==================== EPOCH : 2 ============================
loss:4.602813243865967
loss:4.600244045257568
loss:4.604970455169678
loss:4.606261253356934
l